In [3]:
import os
import string
from glob import glob
import sys
sys.path.insert(0, '../core')

import numpy as np
import pandas as pd
from skimage import io

import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

from load.load_data import dirtyMNISTDataset
from model.cnn import VanillaCNN

In [5]:
model = VanillaCNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=.001)

In [6]:
PATH = '../checkpoint/VanillaCNN_0.pt'
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']


In [7]:
model.eval()

VanillaCNN(
  (network): Sequential(
    (conv2d_0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu_1): ReLU(inplace=True)
    (maxpool2d_2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (dropout2d_3): Dropout2d(p=0.5, inplace=False)
    (conv2d_4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (relu_5): ReLU(inplace=True)
    (maxpool2d_6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (dropout2d_7): Dropout2d(p=0.5, inplace=False)
    (flatten_8): Flatten(start_dim=1, end_dim=-1)
    (linear_9): Linear(in_features=262144, out_features=256, bias=True)
    (relu_10): ReLU(inplace=True)
    (linear_11): Linear(in_features=256, out_features=128, bias=True)
    (relu_12): ReLU(inplace=True)
    (linear_13): Linear(in_features=128, out_features=26, bias=True)
    (sigmoid_14): Sigmoid()
  )
)

In [10]:
dataset = dirtyMNISTDataset(mode='valid', data_path='../data/dirty_mnist/')

In [12]:
dataloader = DataLoader(dataset, batch_size=16)

In [54]:
for idx, sample in enumerate(iter(dataloader)):
    if idx < 10:
        y_true = sample['label']
        y_true = pd.DataFrame(y_true.detach().cpu().numpy().astype('int'), columns=list(string.ascii_lowercase))

        y_pred = model.forward(sample['image'].view(16, 1, 256, 256).float().to(device))
        y_pred = pd.DataFrame(y_pred.detach().cpu().numpy().astype('int'), columns=list(string.ascii_lowercase))

        display(y_true)
        display(y_pred)

,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
0,0,1,0,1,0,1,0,1,1,0,1,0,1,0,0,1,0,1,0,0,0,1,0,1,0,0
1,0,1,0,1,1,0,1,0,0,1,1,0,1,0,0,0,1,1,0,0,0,0,0,0,1,1
2,1,1,0,1,1,1,1,1,0,1,0,1,0,0,0,1,1,0,1,0,0,1,0,1,0,0
3,1,1,1,0,0,1,0,0,0,1,0,0,1,1,1,1,1,0,1,0,0,1,0,0,0,1
4,0,0,0,1,1,0,1,1,0,1,0,0,0,0,0,1,0,0,0,1,0,1,1,1,0,0
5,1,1,0,1,0,1,1,1,0,1,1,0,0,0,1,0,0,0,0,1,1,1,0,0,0,0
6,0,1,0,0,0,1,1,0,1,0,1,0,0,1,1,0,0,1,1,1,1,0,0,0,1,1
7,0,0,1,0,0,1,0,0,0,1,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,1
8,0,0,0,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,1,0,1,1,1,1,1
9,1,1,1,0,0,0,1,0,1,1,0,1,1,1,0,1,0,1,0,0,0,1,1,0,0,0


,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0
5,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0
6,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0
7,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0
8,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0
9,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0


,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
0,0,1,1,1,1,1,1,0,0,1,0,0,0,1,0,0,0,1,1,0,1,0,1,0,1,0
1,0,1,1,1,0,0,0,0,0,1,1,1,0,1,0,1,0,1,0,1,0,1,1,0,1,0
2,0,1,0,0,0,1,0,1,1,1,0,1,1,0,1,0,0,1,0,0,1,1,1,0,0,1
3,0,0,1,1,1,1,1,0,1,1,1,1,0,0,1,0,0,1,0,0,0,0,1,1,1,0
4,1,0,1,1,1,1,0,1,0,1,1,0,1,0,1,0,0,0,0,1,0,1,0,1,0,1
5,1,0,0,1,0,0,0,1,0,0,0,1,0,1,1,0,0,1,1,1,0,0,1,1,1,1
6,0,0,0,0,0,1,1,1,1,0,1,0,0,1,1,0,1,0,1,0,1,1,1,1,0,1
7,0,1,0,1,1,0,1,0,1,1,0,1,0,0,1,0,1,1,0,1,0,1,1,0,1,0
8,0,0,0,0,1,0,1,1,0,1,1,0,0,0,1,0,0,1,0,1,1,1,1,1,0,0
9,0,0,0,1,1,0,1,0,1,1,0,1,0,1,0,0,0,0,0,1,0,1,0,1,1,0


,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0
5,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0
6,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0
7,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0
8,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0
9,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0


,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
0,1,0,1,1,0,0,1,1,0,0,1,0,1,1,0,0,0,1,1,0,0,1,0,1,0,0
1,0,1,1,0,1,0,1,1,1,1,1,0,1,0,0,0,1,0,0,0,0,0,1,1,1,1
2,0,1,1,0,1,1,0,1,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0
3,1,1,1,0,0,1,1,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0
4,1,0,0,0,0,0,1,1,0,1,0,0,0,0,1,0,0,1,0,1,1,0,1,1,1,1
5,0,0,0,1,1,1,0,1,0,1,1,0,0,0,1,1,1,0,0,1,0,1,0,1,0,1
6,0,0,1,0,1,1,1,1,0,0,0,1,1,0,0,0,1,0,1,1,0,0,0,1,0,0
7,0,1,1,0,1,0,1,0,0,0,1,1,1,0,0,1,1,1,0,1,1,0,0,1,0,1
8,1,0,1,0,1,1,1,0,1,1,1,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0
9,0,1,1,1,0,1,1,1,0,0,0,0,1,1,0,0,0,1,1,0,1,1,0,1,1,0


...

,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0
5,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0
6,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0
7,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0
8,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0
9,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0


,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
0,1,0,1,1,0,0,0,0,1,1,1,1,0,0,0,0,1,1,0,1,0,0,0,1,1,1
1,1,1,1,0,1,1,1,1,0,1,0,0,0,0,0,0,0,1,1,1,1,0,1,0,0,1
2,0,1,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0,1,0,1
3,1,0,1,0,1,1,0,0,1,1,0,0,1,0,0,1,1,0,1,0,0,0,0,1,1,1
4,0,0,1,1,0,1,1,0,0,1,1,0,0,0,0,0,1,1,1,1,1,0,0,1,1,0
5,1,1,0,1,1,0,0,1,1,0,1,1,1,0,0,0,1,0,0,0,0,1,0,1,0,1
6,1,1,0,0,0,0,0,1,0,0,1,0,1,1,0,0,0,0,1,0,0,1,0,0,1,1
7,0,0,0,0,0,1,1,0,1,0,1,1,1,0,0,1,0,0,1,1,1,0,1,1,0,0
8,0,0,1,0,1,0,1,0,1,0,1,0,1,0,0,1,0,1,0,0,0,1,1,1,0,0
9,0,1,1,0,1,0,0,0,1,0,1,0,1,0,0,0,1,0,1,1,0,0,1,1,0,0


,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0
5,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0
6,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0
7,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0
8,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0
9,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0


,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
0,0,0,0,1,1,0,1,1,0,1,0,0,1,1,1,1,0,1,1,0,0,0,1,1,0,0
1,0,1,1,1,0,0,0,1,1,1,1,0,0,1,0,1,0,0,1,1,0,0,1,1,0,1
2,1,1,1,1,1,0,1,0,0,0,1,1,0,0,1,0,0,1,1,0,1,0,1,1,0,0
3,1,1,1,1,0,0,1,0,1,0,1,1,1,0,1,1,0,0,0,0,0,0,1,0,1,0
4,0,1,0,0,0,1,1,1,0,0,0,1,0,0,0,1,0,1,1,1,0,1,0,1,0,1
5,1,0,1,1,1,0,1,0,0,1,0,1,1,0,1,1,0,0,0,0,1,0,0,1,0,0
6,0,1,0,0,1,1,0,0,0,1,0,1,1,0,1,0,0,1,1,0,0,0,1,1,0,0
7,0,1,0,0,1,0,1,0,0,0,0,1,1,1,0,0,0,0,1,1,0,1,0,1,0,0
8,1,1,0,0,0,1,0,1,1,1,1,0,1,1,0,0,1,1,0,0,0,0,1,1,1,0
9,0,0,1,0,0,0,1,0,0,1,0,0,0,1,1,0,0,0,0,1,1,1,1,0,0,1


,a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0
5,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0
6,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0
7,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0
8,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0
9,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0
